<a href="https://colab.research.google.com/github/ritika-singh2000/Recommender-System/blob/main/Ritika_Singh__Machine_Learning_Test_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# In this test , I have tried to use 2 methods used for the recomendation system .
# First being content-based filtering technique (Here , I have used TF/IDF model  to model the relationship between different documents within a corpus, this will be useful to find similar posts for a given post).
# Second being colaborating filtering method (Here, I have used Nearest Neighbors algorithm which calculates similarity between users by comparing their ratings on the same item , this will be helpful to recommend posts for a given user).
# I have integrated these into a web app using streamlit .


In [ ]:
# method 1 :  Content-based filtering for recommending posts of similar types
# Importing Libraries
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
posts=pd.read_csv('gdrive/My Drive/posts.csv')
posts.head()

,_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog


In [ ]:
users=pd.read_csv('gdrive/My Drive/users.csv')
users.head()

,_id,name,gender,academics
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate
1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate
2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate
3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate
4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate


In [ ]:
views=pd.read_csv('gdrive/My Drive/views.csv')
views.head()

,user_id,post_id,timestamp
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,2020-06-01T08:10:23.880Z
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z


In [ ]:
print("shape of posts : " , posts.shape)
print("shape of views : " , views.shape)
print("shape of users : " , users.shape)

shape of posts :  (493, 4)
shape of views :  (1449, 3)
shape of users :  (118, 4)


In [ ]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   _id         493 non-null    object
 1   title       493 non-null    object
 2   category    465 non-null    object
 3    post_type  493 non-null    object
dtypes: object(4)
memory usage: 15.5+ KB


In [ ]:
# Filling up the NAN values with empty strings
posts['category'] = posts['category'].fillna('')

In [ ]:
views.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    1449 non-null   object
 1   post_id    1449 non-null   object
 2   timestamp  1449 non-null   object
dtypes: object(3)
memory usage: 34.1+ KB


In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   _id        118 non-null    object
 1   name       118 non-null    object
 2   gender     118 non-null    object
 3   academics  118 non-null    object
dtypes: object(4)
memory usage: 3.8+ KB


In [ ]:
views.describe()


,user_id,post_id,timestamp
count,1449,1449,1449
unique,118,495,1449
top,5d60098a653a331687083238,5ec1fd0974f7660d73aa0fd5,2020-03-30T05:53:20.584Z
freq,230,18,1


In [ ]:
users.describe()

,_id,name,gender,academics
count,118,118,118,118
unique,118,118,3,3
top,5ed237ee76027d35905cc6c5,Shanya,male,undergraduate
freq,1,1,72,68


In [ ]:
posts.describe()


,_id,title,category,post_type
count,493,493,493,493
unique,493,477,232,4
top,5e8381cfa3258347b42f2428,PENCIL RENDERING,Photography,artwork
freq,1,3,81,241


In [ ]:
po = posts.copy()
posts.drop('_id',axis =1 , inplace=True)
posts

,title,category,post_type
0,hello there,Plant Biotechnology,blog
1,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,What is an Operating System ?,Operating Systems,blog
3,Lord Shiva,Drawings,artwork
4,How Competition law evolved?,Competition Laws,blog
...,...,...,...
488,CI/CD using GitHub Actions,Computer Technology|Information Technology,blog
489,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology|Computer Application|Infor...,blog
490,Recommendation Engine,,project
491,Face Recognition using Transfer Learning.,Science; Technology,skill


In [ ]:
# Making the summary  
posts['data'] =  posts['category'] + " " + posts[' post_type'] + " " + posts['title'] 
posts.head(15)

,title,category,post_type,data
0,hello there,Plant Biotechnology,blog,Plant Biotechnology blog hello there
1,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog,Artificial Intelligence|Machine Learning|Infor...
2,What is an Operating System ?,Operating Systems,blog,Operating Systems blog What is an Operating Sy...
3,Lord Shiva,Drawings,artwork,Drawings artwork Lord Shiva
4,How Competition law evolved?,Competition Laws,blog,Competition Laws blog How Competition law evol...
5,Raghavan Committee - 2002.,Competition Laws,blog,Competition Laws blog Raghavan Committee - 2002.
6,Let's discuss some Case laws!,Competition Laws,blog,Competition Laws blog Let's discuss some Case ...
7,Forms of Cartel.,Competition Laws,blog,Competition Laws blog Forms of Cartel.
8,Biodiversity,Eco System,blog,Eco System blog Biodiversity
9,Economic policy,Economic Policies,blog,Economic Policies blog Economic policy


In [ ]:
# Removing the punctuation marks
posts['data'] = posts['data'].str.replace('[^\w\s]',' ')

In [ ]:

posts.head(20)

,title,category,post_type,data
0,hello there,Plant Biotechnology,blog,Plant Biotechnology blog hello there
1,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog,Artificial Intelligence Machine Learning Infor...
2,What is an Operating System ?,Operating Systems,blog,Operating Systems blog What is an Operating Sy...
3,Lord Shiva,Drawings,artwork,Drawings artwork Lord Shiva
4,How Competition law evolved?,Competition Laws,blog,Competition Laws blog How Competition law evol...
5,Raghavan Committee - 2002.,Competition Laws,blog,Competition Laws blog Raghavan Committee 2002
6,Let's discuss some Case laws!,Competition Laws,blog,Competition Laws blog Let s discuss some Case ...
7,Forms of Cartel.,Competition Laws,blog,Competition Laws blog Forms of Cartel
8,Biodiversity,Eco System,blog,Eco System blog Biodiversity
9,Economic policy,Economic Policies,blog,Economic Policies blog Economic policy


In [ ]:
posts.to_csv("final_posts_01", sep='\t')

In [ ]:
tfv = TfidfVectorizer(min_df = 0 , max_features  = None , strip_accents = 'unicode' , analyzer = 'word', ngram_range=(1,3) , stop_words = 'english')  

In [ ]:
# Fit and transforming the 'data' text 
tfv_matrix = tfv.fit_transform(posts['data'])
tfv_matrix

<493x4367 sparse matrix of type '<class 'numpy.float64'>'
	with 7664 stored elements in Compressed Sparse Row format>

In [ ]:
tfv_matrix.shape

(493, 4367)

In [ ]:
# getting the scores
sig = sigmoid_kernel(tfv_matrix , tfv_matrix)

In [ ]:
sig[0]

array([0.76169031, 0.76159492, 0.76159527, 0.76159416, 0.76159503,
       0.76159505, 0.76159495, 0.76159519, 0.76159556, 0.76159505,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159526, 0.76159416, 0.76159506, 0.76159416,
       0.76159416, 0.76159416, 0.76159483, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.7615951 , 0.76159416,
       0.76159416, 0.76159504, 0.76159416, 0.76159504, 0.76159416,
       0.76159488, 0.76159497, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.7615951 , 0.76159474,
       0.76159416, 0.76159416, 0.76159416, 0.76159485, 0.76159525,
       0.76159495, 0.76159471, 0.7615949 , 0.76159484, 0.76159489,
       0.76159488, 0.761595  , 0.76159477, 0.76159479, 0.7615949 ,
       0.76159481, 0.761595  , 0.76159416, 0.76159521, 0.7615952 ,
       0.76159416, 0.76159494, 0.76159483, 0.76159486, 0.76159496,
       0.76159476, 0.76159474, 0.76159484, 0.76159489, 0.76159

In [ ]:
# getting the indices values for each post
indices = pd.Series(posts.index , index = posts['title'] ).drop_duplicates()
indices

title
hello there                                                              0
Ml and AI                                                                1
What is an Operating System ?                                            2
Lord Shiva                                                               3
How Competition law evolved?                                             4
                                                                      ... 
CI/CD using GitHub Actions                                             488
Configure Docker with Django; PostgreSQL; Pg-admin & Elasticsearch     489
Recommendation Engine                                                  490
Face Recognition using Transfer Learning.                              491
Configure CI/CD Pipeline in GitLab and deployment to server via SSH    492
Length: 493, dtype: int64

In [ ]:
# p = posts.copy()
# Function used to print the top 10 recommendations
def recommendation_prediction(title , sig=sig):
  idx = indices[title]
  sig_scores = list(enumerate(sig[idx]))
  sig_scores = sorted(sig_scores , key = lambda x : x[1] , reverse =  True)
  sig_scores = sig_scores[1:11]
  movies_indies = [i[0] for i in sig_scores]
  return (posts['title'].iloc[movies_indies] , sig_scores) 

In [ ]:
x ='Ml and AI'
recommendation_prediction(x)

(122                              Artificial Intelligence
 128                      What is Information technology?
 224    10 Best Artificial Intelligence (AI) Courses O...
 291             8 Best Machine Learning Courses for 2020
 16                                                   MIS
 110                                             Ethernet
 170                                         Types Of AI.
 486                                          Learning...
 456                   Recommend Systems Machine Learning
 450               Machine Learning”&“Operations" (MlOps)
 Name: title, dtype: object,
 [(122, 0.7616170480705129),
  (128, 0.761616939172401),
  (224, 0.7616140070230293),
  (291, 0.7616131111283064),
  (16, 0.7616127386143149),
  (110, 0.7616127386143149),
  (170, 0.7616126897942919),
  (486, 0.7616121973946585),
  (456, 0.7616121849618491),
  (450, 0.7616120758351735)])

In [ ]:
# using the linear_kernel
from sklearn.metrics.pairwise import linear_kernel
li = linear_kernel(tfv_matrix , tfv_matrix)
li[0]

array([1.        , 0.00795888, 0.01154026, 0.        , 0.0090451 ,
       0.00931955, 0.00825867, 0.01077362, 0.01455042, 0.00933305,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.0114697 , 0.        , 0.00936524, 0.        ,
       0.        , 0.        , 0.00698242, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00985546, 0.        ,
       0.        , 0.00914785, 0.        , 0.00921591, 0.        ,
       0.00755013, 0.00845433, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00985546, 0.0060355 ,
       0.        , 0.        , 0.        , 0.00725657, 0.01142709,
       0.00820983, 0.00576162, 0.00773842, 0.00714836, 0.00761643,
       0.00751277, 0.00875371, 0.00635404, 0.00658688, 0.00771889,
       0.00679696, 0.00875371, 0.        , 0.01099062, 0.0108302 ,
       0.        , 0.0081122 , 0.00697659, 0.00729073, 0.00838358,
       0.00631815, 0.00602209, 0.00709209, 0.00759055, 0.00735

In [ ]:
li_indices = pd.Series(posts.index , index = posts['title'] ).drop_duplicates()
li_indices

title
hello there                                                              0
Ml and AI                                                                1
What is an Operating System ?                                            2
Lord Shiva                                                               3
How Competition law evolved?                                             4
                                                                      ... 
CI/CD using GitHub Actions                                             488
Configure Docker with Django; PostgreSQL; Pg-admin & Elasticsearch     489
Recommendation Engine                                                  490
Face Recognition using Transfer Learning.                              491
Configure CI/CD Pipeline in GitLab and deployment to server via SSH    492
Length: 493, dtype: int64

In [ ]:
x ='Formation Of Trade Union'
recommendation_prediction(x,li)

(211                               JUSTICE FOR NORTH EAST
 70                               Human Rights and Duties
 71                    Concept of Human Rights and Duties
 35                             Human Resource Management
 226                        Are We Alone In The Universe?
 171               Accounting As A Financial Institution.
 125    Preparation and Presentation of Company Final ...
 434                                   HUMAN ILLUSTRATION
 433           HUMAN ILLUSTRATION ( Background of a room)
 432          HUMAN ILLUSTRATION (With Galaxy Background)
 Name: title, dtype: object,
 [(211, 0.3827831420030752),
  (70, 0.2682293860079221),
  (71, 0.25566048140167885),
  (35, 0.05172555329831923),
  (226, 0.03711848791708398),
  (171, 0.03701717325216221),
  (125, 0.03412412647418128),
  (434, 0.028365996546882967),
  (433, 0.02365274229482719),
  (432, 0.023415477597527063)])

In [ ]:
#method 2 : colaborative filtering for the recommendation for the user
posts=pd.read_csv('gdrive/My Drive/posts.csv')
# renaming the column '_id' 
posts.rename(columns={'_id':'post_id'}, inplace = True)
posts.head()

,post_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog


In [ ]:
views=pd.read_csv('gdrive/My Drive/views.csv')
views.describe()

,user_id,post_id,timestamp
count,1449,1449,1449
unique,118,495,1449
top,5d60098a653a331687083238,5ec1fd0974f7660d73aa0fd5,2020-03-30T05:53:20.584Z
freq,230,18,1


In [ ]:
users=pd.read_csv('gdrive/My Drive/users.csv')
users.head(5)

,_id,name,gender,academics
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate
1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate
2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate
3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate
4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate


In [ ]:
print("shape of posts : " , posts.shape)
print("shape of views : " , views.shape)
print("shape of users : " , users.shape)

shape of posts :  (493, 4)
shape of views :  (1449, 3)
shape of users :  (118, 4)


In [ ]:
# constructing a new dataframe with count of total number of views of a post.
post_total_viewsCount = views['post_id'].value_counts()
post_total_viewsCount = pd.DataFrame({'post_id':post_total_viewsCount.index, 'total_views':post_total_viewsCount.values})
post_total_viewsCount

,post_id,total_views
0,5ec1fd0974f7660d73aa0fd5,18
1,5ecd5d417023451e662235c5,16
2,5ecc1d4eeaff6b0c3a58a565,14
3,5eb2c11210426255a7aaa052,14
4,5ecce8a5eaff6b0c3a58a5e9,14
...,...,...
490,5e948786a3258347b42f287e,1
491,5e8bfacca3258347b42f261b,1
492,5e3fcb74eab55d319938a806,1
493,5d80ecfd6c53455f896e601a,1


In [ ]:
total_views_list = post_total_viewsCount.total_views.to_list()
len(post_total_viewsCount)

495

In [ ]:
# Merging the dataframe posts with post_total_viewsCount
posts = posts.merge(post_total_viewsCount, on = 'post_id')
posts

,post_id,title,category,post_type,total_views
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog,1
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog,1
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog,2
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork,3
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog,2
...,...,...,...,...,...
488,5ed3791976027d35905ccab6,CI/CD using GitHub Actions,Computer Technology|Information Technology,blog,2
489,5ed3820f76027d35905ccac8,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology|Computer Application|Infor...,blog,4
490,5ed3ef4cbd514d602c1530f2,Recommendation Engine,NaN,project,1
491,5ed415c6bd514d602c15312d,Face Recognition using Transfer Learning.,Science; Technology,skill,2


In [ ]:
# getting the Individual Count of views , using a groupby method
individual_views_df = (views.
     groupby(by = ['user_id','post_id'])['timestamp'].
     count().
     reset_index().
     rename(columns = {'timestamp': 'Individual Count of views '})
    #  [['user_id', 'Individual Count']]
    )
individual_views_df

,user_id,post_id,Individual Count of views
0,5d60098a653a331687083238,5d62abaa65218653a132c956,1
1,5d60098a653a331687083238,5d6d39567fa40e1417a4931c,1
2,5d60098a653a331687083238,5dbc631f99cbb90e4339c7fd,1
3,5d60098a653a331687083238,5dc065ca24b883670268772f,1
4,5d60098a653a331687083238,5ddeb6e80eb5e25a8a07f065,1
...,...,...,...
1392,5ed3748576027d35905ccaab,5ed3791976027d35905ccab6,1
1393,5ed3748576027d35905ccaab,5ed3820f76027d35905ccac8,1
1394,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,1
1395,5ed3cbcbbd514d602c1530a7,5ed1ff0276027d35905cc60d,1


In [ ]:
individual_views_df.shape

(1397, 3)

In [ ]:
individual_views_df.describe()

,Individual Count of views
count,1397.000000
mean,1.037223
std,0.217541
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,5.000000


In [ ]:
# Final dataframe
df = pd.merge(individual_views_df,posts,on='post_id')
df

,user_id,post_id,Individual Count of views,title,category,post_type,total_views
0,5d60098a653a331687083238,5d62abaa65218653a132c956,1,hello there,Plant Biotechnology,blog,1
1,5d60098a653a331687083238,5d6d39567fa40e1417a4931c,1,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog,1
2,5d60098a653a331687083238,5dbc631f99cbb90e4339c7fd,1,Calligraphy,Drawings|Calligraphy,artwork,1
3,5d60098a653a331687083238,5dc065ca24b883670268772f,1,Colours of pushkar.,Photography,artwork,1
4,5d60098a653a331687083238,5ddeb6e80eb5e25a8a07f065,1,Library Managment System: Software Requirement...,NaN,project,2
...,...,...,...,...,...,...,...
1390,5ecb979eeaff6b0c3a58a4f0,5ed415c6bd514d602c15312d,1,Face Recognition using Transfer Learning.,Science; Technology,skill,2
1391,5ed0defa76027d35905cc2de,5ed415c6bd514d602c15312d,1,Face Recognition using Transfer Learning.,Science; Technology,skill,2
1392,5ecf6e6576027d35905cbeec,5ed0007a76027d35905cc0ea,1,A solar-powered microbrewery has design-led en...,Artistic design,artwork,1
1393,5ed0defa76027d35905cc2de,5ed23d4276027d35905cc798,1,Energy.,Calligraphy|Typography|Pen and ink,artwork,1


In [ ]:
users_2 = users[['_id' ,'name']]
users_2.rename(columns={'_id' :'user_id'}, inplace = True)
my_df = pd.merge(df,users_2,on='user_id' ,how='outer')
my_df

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,user_id,post_id,Individual Count of views,title,category,post_type,total_views,name
0,5d60098a653a331687083238,5d62abaa65218653a132c956,1,hello there,Plant Biotechnology,blog,1,Nivesh Singh Chauhan
1,5d60098a653a331687083238,5d6d39567fa40e1417a4931c,1,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog,1,Nivesh Singh Chauhan
2,5d60098a653a331687083238,5dbc631f99cbb90e4339c7fd,1,Calligraphy,Drawings|Calligraphy,artwork,1,Nivesh Singh Chauhan
3,5d60098a653a331687083238,5dc065ca24b883670268772f,1,Colours of pushkar.,Photography,artwork,1,Nivesh Singh Chauhan
4,5d60098a653a331687083238,5ddeb6e80eb5e25a8a07f065,1,Library Managment System: Software Requirement...,NaN,project,2,Nivesh Singh Chauhan
...,...,...,...,...,...,...,...,...
1390,5e26bb482a37d20505da2c10,5e4d359cf5561b1994c8e424,1,Business skills,Business|Business Skills,blog,1,Nachiket Khatri
1391,5e78ce84cfc8b713f5ac7cee,5e78d6dccfc8b713f5ac7cf4,1,Childhood Sketch,Drawings|Watercolours,artwork,1,Fauziya Shaikh
1392,5e8437f6a3258347b42f243d,5e901b98a3258347b42f2708,1,Must-Visit Hill Stations in Maharashtra,Geography|Indian Geography,blog,3,Shivani Gaba
1393,5e8437f6a3258347b42f243d,5e908602a3258347b42f275b,1,Creativity During Quarantine?,Literature|Stories,blog,1,Shivani Gaba


In [ ]:
df=my_df.copy()

In [ ]:
df.describe()

,Individual Count of views,total_views
count,1395.000000,1395.000000
mean,1.037276,5.384229
std,0.217692,4.014862
min,1.000000,1.000000
25%,1.000000,2.000000
50%,1.000000,4.000000
75%,1.000000,8.000000
max,5.000000,18.000000


In [ ]:
# Setting a threshold value and cosidering the values only beyond the following threshold value
views_threshold = 4
rating_populatity =  df.query('total_views >=@views_threshold')
rating_populatity.head()

,user_id,post_id,Individual Count of views,title,category,post_type,total_views,name
7,5d60098a653a331687083238,5e2d36ebc85ab714a7da66d1,1,CONTOURING CLAY,Sculptures|Artistic design,artwork,5,Nivesh Singh Chauhan
12,5d60098a653a331687083238,5e7304761b24db0468e90ce7,1,Textile And Apparel Testing,Fashion Desigining|Fashion Textile|Fashion Man...,blog,4,Nivesh Singh Chauhan
29,5d60098a653a331687083238,5e7de250a3258347b42f210a,1,Top 10 big benefits of having a digital market...,Digital Marketing,skill,11,Nivesh Singh Chauhan
33,5d60098a653a331687083238,5e7e66e7a3258347b42f213c,1,Netflix: As a Case Study,Mass Media|Videography,blog,6,Nivesh Singh Chauhan
34,5d60098a653a331687083238,5e7f39a3a3258347b42f2151,1,The Meenamma,Drawings|Painting|Visual Arts|Artistic design|...,artwork,9,Nivesh Singh Chauhan


In [ ]:
rating_populatity.shape

(785, 8)

In [ ]:
# Creating a pivot table 
df_pivot = df.pivot_table(index = 'name' , columns = 'title' , values  = 'Individual Count of views ').fillna(0)
df_pivot.head()

title,'The Virtual ME',10 Best Artificial Intelligence (AI) Courses Online for 2020,10 Best UI UX Design Course & Certification [2020],100 Free University Courses Online [2020],20 Best Data Science Certification & Courses for 2020,21 day Lock down. Quarantine,3D composition.,5 Best Computer Networking Courses & Certification [2020],5 Best Productivity Web Apps For Students.,5 points to consider while choosing a pathology lab,6 Best + Free PLC Programming Training & Course [2020 UPDATED],7 Best Python Data Science Courses & Certification [2020],7 Steps To Prevent COVID-19,7 Steps To Stay Safe From Corona Virus,8 Best Machine Learning Courses for 2020,8 Free Data Science Courses & Tutorials [2020],A solar-powered microbrewery has design-led energy in Sweden,A stone bridge on the full moon night,AUDIT REMARK,AWS CLI Setup in Mac,AWS services and how to launch OS on AWS Cloud,Abacus,Acceptance!,Accounting As A Financial Institution.,Advantages of Online Shopping Over Offline Shopping,Aesthetic,Aesthetic.,Aesthetics,Alone,Always Remember !!!,Always do Right 👍🏻,Analysis Of Successful Companies For the Past 5 years,App Development,Applications of mobile computing,Are We Alone In The Universe?,Armstrong No.(C# .Net),Art,Art Expo 2020,Artificial Intelligence,Avengers Theme - Acoustic guitar,...,What is an Operating System ?,What sports will look like in the future,Why Do Millennials Change Jobs?,Wildlife Creation.,Will AI replace humans intellectual work?,Will IPL happen this year,WisdomLeaf,Women power,Women: goddess,Work From Home: Working Under Corona's Effect,Workforce Diversity as a Challenge,Xbox Series X To Launch With 'Thousands' of Games,You are late.,You are one in a kind!,You know if you've been here.,You; are enough!,Your reaction can change your destination.,Zero-Waste Lifestyle,aloof,based on your imagination,deep thoughts,early mornings with Gang,engrossed,filtered,freeze,gripping,hello there,innocence,insect,monument,peace,portraying goddess,screw town,screw2,silence,street art; Jaisalmer,trekking,upside down,आमची मुंबाई !!!,🍉 Watermelon lemon cooler
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Aanjan Prasath G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aayushree Khokhar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abhinav Mishra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abhishank Gupta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abhishek Kumar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

final_df = csr_matrix(df_pivot.values)
model_knn = NearestNeighbors(metric = 'cosine' , algorithm = 'brute')
model_knn.fit(final_df)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
query_index = np.random.choice(final_df.shape[0])
print(query_index)
distances , indices  = model_knn.kneighbors(df_pivot.iloc[query_index,:].values.reshape(1,-1) , n_neighbors=10)

93


In [ ]:
df_pivot.to_csv("users_pivot_table_01")
!cp pivot_table_01.csv "drive/My Drive/"
df_pivot.head()

cp: cannot stat 'pivot_table_01.csv': No such file or directory


title,'The Virtual ME',10 Best Artificial Intelligence (AI) Courses Online for 2020,10 Best UI UX Design Course & Certification [2020],100 Free University Courses Online [2020],20 Best Data Science Certification & Courses for 2020,21 day Lock down. Quarantine,3D composition.,5 Best Computer Networking Courses & Certification [2020],5 Best Productivity Web Apps For Students.,5 points to consider while choosing a pathology lab,6 Best + Free PLC Programming Training & Course [2020 UPDATED],7 Best Python Data Science Courses & Certification [2020],7 Steps To Prevent COVID-19,7 Steps To Stay Safe From Corona Virus,8 Best Machine Learning Courses for 2020,8 Free Data Science Courses & Tutorials [2020],A solar-powered microbrewery has design-led energy in Sweden,A stone bridge on the full moon night,AUDIT REMARK,AWS CLI Setup in Mac,AWS services and how to launch OS on AWS Cloud,Abacus,Acceptance!,Accounting As A Financial Institution.,Advantages of Online Shopping Over Offline Shopping,Aesthetic,Aesthetic.,Aesthetics,Alone,Always Remember !!!,Always do Right 👍🏻,Analysis Of Successful Companies For the Past 5 years,App Development,Applications of mobile computing,Are We Alone In The Universe?,Armstrong No.(C# .Net),Art,Art Expo 2020,Artificial Intelligence,Avengers Theme - Acoustic guitar,...,What is an Operating System ?,What sports will look like in the future,Why Do Millennials Change Jobs?,Wildlife Creation.,Will AI replace humans intellectual work?,Will IPL happen this year,WisdomLeaf,Women power,Women: goddess,Work From Home: Working Under Corona's Effect,Workforce Diversity as a Challenge,Xbox Series X To Launch With 'Thousands' of Games,You are late.,You are one in a kind!,You know if you've been here.,You; are enough!,Your reaction can change your destination.,Zero-Waste Lifestyle,aloof,based on your imagination,deep thoughts,early mornings with Gang,engrossed,filtered,freeze,gripping,hello there,innocence,insect,monument,peace,portraying goddess,screw town,screw2,silence,street art; Jaisalmer,trekking,upside down,आमची मुंबाई !!!,🍉 Watermelon lemon cooler
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Aanjan Prasath G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aayushree Khokhar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abhinav Mishra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abhishank Gupta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abhishek Kumar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print("Recomendation for  User:")
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('{}'.format(df_pivot.index[query_index]))
        # print(query_index)
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, df_pivot.columns[indices.flatten()[i]], distances.flatten()[i]))

Recomendation for  User:
Shreya maji
1: AWS services and how to launch OS on AWS Cloud, with distance of 0.7642977396044841:
2: BASIC STRUCTURE DOCTRINE, with distance of 0.8578661890962597:
3: 5 Best Computer Networking Courses & Certification [2020], with distance of 0.8796141469142308:
4: Database management system(DBMS), with distance of 0.882148869802242:
5: C#(.NET), with distance of 0.8867722965855405:
6: Communication, with distance of 0.8908910548820038:
7: Delegates(.net), with distance of 0.8951354989813847:
8: A stone bridge on the full moon night, with distance of 0.9394772467331197:
9: Business skills, with distance of 0.9631332490927113:
